### Example using MF6 flexible discretisation

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon, LineString
import flopy
from flopy.discretization import VertexGrid
from flopy.utils.triangle import Triangle as Triangle
from flopy.utils.voronoi import VoronoiGrid
%matplotlib inline
from collections import OrderedDict

### Set up project paths

In [ ]:
triExeName = './exe/triangle.exe'
mf6ExeName = './exe/mf6.exe'
name = 'mf6_example'
# For this example, we will set up a model workspace.
# Model input files and output files will reside here.
workspace = './modelfiles'
if not os.path.exists(workspace):
    os.makedirs(workspace)

### Define domains
The model domain is a low K area and there is a high K zone conceptualized as a rectangle in the center left part of the model. 

In [ ]:
Lx = 5000.
Ly = 3000.
domain = [(0, 0),(Lx,0), (Lx,Ly), (0, Ly)]
hiK_zone = [(1500, 1400), (2300, 600), (2800, 620),(3500, 1400), (2300, 1800), (1800, 1700), (1500,1600)]

# Create a Polygon
domain_poly = Polygon(domain)
hiK_zone_poly = Polygon(hiK_zone)

### Triangle mesh creation

In [ ]:
tri = Triangle(angle=32, model_ws=workspace, exe_name=triExeName)
tri.add_polygon(domain)
tri.add_polygon(hiK_zone)
tri.add_region((1000,200),0,maximum_area=50000) #coarse discretization in domain Default = 50000
tri.add_region((2500,1500),0,maximum_area=10000) #fine discretization in hiK_zone Default = 10000
tri.build()

cell2d = tri.get_cell2d()     # cell info: id,x,y,nc,c1,c2,c3 (list of lists)
vertices = tri.get_vertices()
xcyc = tri.get_xcyc()   
vgrid = flopy.discretization.VertexGrid(vertices=vertices, 
                                             cell2d=cell2d, 
                                             ncpl = len(cell2d), 
                                             nlay = 1)  
gi = flopy.utils.GridIntersect(vgrid)

### Voronoi mesh creation

In [ ]:
vor = VoronoiGrid(tri)
vorvertices = vor.get_disv_gridprops()['vertices']
vorcell2d = vor.get_disv_gridprops()['cell2d']
vorxcyc = []
for cell in vorcell2d:
    vorxcyc.append((cell[1],cell[2]))
vorvgrid = flopy.discretization.VertexGrid(vertices=vorvertices, 
                                             cell2d=vorcell2d, 
                                             ncpl = len(vorcell2d), 
                                             nlay = 1)  
vorgi = flopy.utils.GridIntersect(vorvgrid)

### Generate mesh

The next step is the mesh representation representation and a identification of the boundaries create from the mesh generation

In [ ]:
fig = plt.figure(figsize=(5, 7))
ax = plt.subplot(2, 1, 1, aspect='equal')
tri.plot(edgecolor='gray', lw=0.5, ax=ax)
ax.plot(*hiK_zone_poly.exterior.xy, color='red', lw=1.5)
for i in xcyc: 
    ax.plot(i[0], i[1], 'o', color = 'green', ms = 0.5)  
ax.set_ylabel('y (m)')
ax.set_title(f'Triangular Mesh, ncell = {len(cell2d)}')


ax = plt.subplot(2, 1, 2, aspect='equal')
vor.plot(edgecolor='gray', lw=0.5, ax=ax)
ax.plot(*hiK_zone_poly.exterior.xy, color='red', lw=1.5)
for i in vorxcyc: 
    ax.plot(i[0], i[1], 'o', color = 'green', ms = 0.5)  
ax.set_title(f'Voronoi Mesh, ncell = {len(vorcell2d)}')
ax.set_xlabel('x (m)')
ax.set_ylabel('y (m)')

### Identify important cells

In [ ]:
# West head boundary
west = [(0, 0), (0, Ly)]
west_ls = LineString(west)
west_cells = gi.intersects(west_ls)["cellids"]
vor_west_cells = vorgi.intersects(west_ls)["cellids"]

# East head boundary
east = [(Lx, 0), (Lx, Ly)]
east_ls = LineString(east)
east_cells = gi.intersects(east_ls)["cellids"]
vor_east_cells = vorgi.intersects(east_ls)["cellids"]

# HiK cells
hiK_cells = []
for i in range(len(cell2d)):
    point = Point(cell2d[i][1],cell2d[i][2])
    if point.within(hiK_zone_poly):
        hiK_cells.append(cell2d[i][0])

vor_hiK_cells = []
for i in range(len(vorcell2d)):
    point = Point(vorcell2d[i][1],vorcell2d[i][2])
    if point.within(hiK_zone_poly):
        vor_hiK_cells.append(vorcell2d[i][0])

print('Triangulated grid hiK cells:', len(hiK_cells))
print('Voronoi grid hiK cells:', len(vor_hiK_cells))

### Plot special cells

In [ ]:
def plot_head_boundary(cell2d, west_cells, east_cells, hiK_cells, vgrid):

    fig = plt.figure(figsize=(7,5))
    ax = fig.add_subplot(1, 1, 1, aspect="equal")
    ax.set_title('Head boundary')
        
    pmv = flopy.plot.PlotMapView(ax = ax, modelgrid=vgrid)
    pmv.plot_grid(color = 'gray', lw = 0.4)

    ibd = np.ones((len(cell2d),), dtype=int)  # Initialize ibd wi
    for cell in west_cells: 
        ax.plot(cell2d[cell][1], cell2d[cell][2], "o", color = 'red', ms = 1)
        ibd[cell] = 2  # Set west cells to 2th ones
    for cell in east_cells: 
        ax.plot(cell2d[cell][1], cell2d[cell][2], "o", color = 'red', ms = 1)
        ibd[cell] = 2  # Set east cells to 2
    for cell in hiK_cells: 
        ax.plot(cell2d[cell][1], cell2d[cell][2], "o", color = 'red', ms = 1)
        ibd[cell] = 3  # Set hiK cells to 3
    p = pmv.plot_array(ibd, alpha = 0.6)
        
plot_head_boundary(cell2d, west_cells, east_cells, hiK_cells, vgrid)
plot_head_boundary(vorcell2d, vor_west_cells, vor_east_cells, vor_hiK_cells, vorvgrid)

### Set up parameters

In [ ]:
# Spatial discretisation
nlay = 1
top = 100.
bottom = 0.

# Time discretisation
nper = 3          # 3 stress periods
perlen1 = 1       # Length of 1st period (days) # Steady state
perlen2 = 365     # Length of 2nd period (days)
perlen3 = 365     # Length of 3rd period (days)
nts1 = 1          # Numer of time-steps 1st period
nts2 = 12         # Numer of time-steps 2nd period
nts3 = 12         # Numer of time-steps 3rd period
perioddata=[(perlen1, nts1, 1.0), (perlen2, nts2, 1.0), (perlen3, nts3, 1.0)]

# Hydraulic conductivity and Ss
k_global = 0.1 # K in model domain
k_hiK = 100   # K in high K zone

# Setting k values for each layer
k = [k_global for i in range(len(cell2d))]
for i in hiK_cells: k[i] = k_hiK

# Storativity
sy = 0.2
ss = 0.0001

# Specified head boundaries
h_west = 90.
h_east = 80.

# Initial conditions
ic_global = 90.

# Specified head package. Don't need to do the ibound=-1 thing in MF6 
chd_rec = []
for icpl in west_cells: chd_rec.append([(0, icpl), h_west])
for icpl in east_cells: chd_rec.append([(0, icpl), h_east])


In [ ]:
np.unique(k, return_counts=True)

In [ ]:
fig = plt.figure(figsize=(7, 7))
ax = plt.subplot(1, 1, 1, aspect='equal')
ax.plot(*hiK_zone_poly.exterior.xy, color='red', lw=1.5)
mapview = flopy.plot.PlotMapView(modelgrid=vgrid, ax=ax)#, layer = layer)
plan = mapview.plot_array(a = k, cmap='Spectral', alpha=0.8)
mapview.plot_grid(lw = 0.2, color = 'black') 
ax.set_title('Hydraulic conductivity')
plt.colorbar(plan, shrink = 0.4)

##### Prepare Observations (OBS) package

In [ ]:
# Finding cell id of obs bores
obs_coords = [[1000,1500], [1000,600], [2500,600]]
obs_cells = []
obs_total = len(obs_coords)

obs_dist = np.ones((obs_total, len(cell2d)))
for j in range(obs_total):
    for i in range(len(cell2d)):
        obs_dist[j][i] = np.sqrt((cell2d[i][1] - obs_coords[j][0])**2 + (cell2d[i][2] - obs_coords[j][1])**2)
    obs_cells.append(np.argmin(obs_dist[j][:]))

print(obs_coords)
print(obs_cells)

##### Prepare Well (WEL) package

In [ ]:
# Finding cell id of most centred cell
dist_from_centre = []
for i in range(len(cell2d)):
    dist_from_centre.append(np.sqrt((cell2d[i][1] - Lx/2)**2 + (cell2d[i][2] - Ly/2)**2))
pump1_cell = int(np.argmin(dist_from_centre))
pump1_coords = (cell2d[pump1_cell][1],cell2d[pump1_cell][2])
print('The pump is located in cell %i at coordinates %s' %(pump1_cell,pump1_coords))

layer_loc = 0   # layer pumping well in

pumping_rate = -1000.0
wel_sp1 = [[(layer_loc, pump1_cell), 0.0]]
wel_sp2 = [[(layer_loc, pump1_cell), pumping_rate]]
wel_sp3 = [[(layer_loc, pump1_cell), 0.]]
well_period_data = {0: wel_sp1, 1: wel_sp2, 2: wel_sp3}


### Write the packages

In [ ]:
# This sets up the Simulation.
sim = flopy.mf6.MFSimulation(sim_name='sim', exe_name=mf6ExeName, version='mf6', sim_ws=workspace)

# Time discretisation (TDIS)
tdis = flopy.mf6.modflow.mftdis.ModflowTdis(sim, pname='tdis', time_units='DAYS', nper=nper, perioddata=perioddata)

# This saves the flow model itself. You can even have multiple flow models in one simulation.
gwf = flopy.mf6.ModflowGwf(sim, modelname=name, save_flows=True)

# This is the solver package (IMS)
ims = flopy.mf6.ModflowIms(sim, print_option='SUMMARY', complexity='complex', outer_dvclose=1.e-6, inner_dvclose=1.e-6)

# Discretisation by Vertices DISV (DIS is MF-2005 equivalent, DISU is for fancy stuff)
disv = flopy.mf6.ModflowGwfdisv(gwf, length_units='METERS',
                               nlay=nlay, ncpl=len(cell2d), nvert=len(vertices),
                               top=top, botm=bottom, 
                               vertices=vertices, cell2d=cell2d)

# NPF is like MF2005's LPF. Stores Hydraulic properties.
npf = flopy.mf6.ModflowGwfnpf(gwf, k=k, save_flows=True, xt3doptions = True)

# Initial conditions (IC)
ic = flopy.mf6.ModflowGwfic(gwf, strt=ic_global)

# Specified head (CHD)
chd = flopy.mf6.ModflowGwfchd(gwf, stress_period_data=chd_rec, save_flows=True)

# Well Package (WEL) 
wel = flopy.mf6.ModflowGwfwel(gwf, stress_period_data=well_period_data)

# Storage Package (STO)
sto = flopy.mf6.modflow.mfgwfsto.ModflowGwfsto(gwf, 
                                               iconvert=0, # Force as confined
                                               sy = sy,
                                               ss=ss, 
                                               steady_state={0: True},
                                               transient={1: True, 2: True},
                                               )

# Output Control (OC) - you can decide what you want to save
oc = flopy.mf6.ModflowGwfoc(gwf,
                            budget_filerecord='{}.cbc'.format(name),
                            head_filerecord='{}.hds'.format(name),
                            saverecord=[('HEAD', 'ALL'),
                                        ('BUDGET', 'ALL')],
                            printrecord=[('HEAD', 'LAST'),
                                         ('BUDGET', 'LAST')])

In [ ]:
# Write and Run model!
sim.write_simulation(silent = False)  
success, buff = sim.run_simulation(silent = False) 

### Post-processing of model results

In [ ]:
# Import results
fname = os.path.join(workspace, name + '.hds')
hds = flopy.utils.binaryfile.HeadFile(fname)

heads_all = hds.get_alldata()
h1 = hds.get_data(kstpkper=(0, 0))[0][0] # Period 1
h2 = hds.get_data(kstpkper=(0, 1))[0][0] # Period 2
h3 = hds.get_data(kstpkper=(0, 2))[0][0] # Period 3

np.set_printoptions(precision=2)
#Print statistics
print('Head statistics Start Period 3')
print('  min: ', h3.min())
print('  max: ', h3.max())
print(heads_all.shape)

### Plot model results

In [ ]:
# This chunk is to get ready for contouring
from scipy.interpolate import griddata
x, y = tri.get_xcyc()[:,0],tri.get_xcyc()[:,1]
xG = np.linspace(x.min(),x.max(),100)
yG = np.linspace(y.min(),y.max(),100)
X, Y = np.meshgrid(xG, yG)

# Look how we can make a plotting function and use it many times
def plot_head(array, title):
    fig = plt.figure(figsize=(7, 7))
    ax = plt.subplot(1, 1, 1, aspect='equal')
    ax.plot(pump1_coords[0],pump1_coords[1], color = 'black', marker = 'o', markersize = 9)
    ax.plot(obs_coords[0][0],obs_coords[0][1], color = 'blue', marker = 'o', markersize = 9)
    ax.plot(obs_coords[1][0],obs_coords[1][1], color = 'blue', marker = 'o', markersize = 9)
    ax.plot(obs_coords[2][0],obs_coords[2][1], color = 'blue', marker = 'o', markersize = 9)
    ax.annotate("Obs1", obs_coords[0], size = 11, xytext=(5, 5), textcoords='offset points')
    ax.annotate("Obs2", obs_coords[1], size = 11, xytext=(5, 5), textcoords='offset points')
    ax.annotate("Obs3", obs_coords[2], size = 11, xytext=(5, 5), textcoords='offset points')
    ax.plot(*hiK_zone_poly.exterior.xy, color='red', lw=1.5)
    mapview = flopy.plot.PlotMapView(modelgrid=vgrid, ax=ax)#, layer = layer)
    plan = mapview.plot_array(a = array, cmap='Spectral', alpha=0.8, vmin = 70, vmax = 90)
    mapview.plot_grid(lw = 0.2, color = 'black') 

    Ti = griddata((x, y), array, (X, Y), method='cubic')
    levels = np.arange(50,90,2)
    CS = ax.contour(X,Y,Ti,levels,colors='Blue', linewidths=1.0)
    ax.clabel(CS, inline=1, fontsize=10, fmt='%1.1f')

    ax.set_title(title)
    plt.colorbar(plan, shrink = 0.4)


plot_head(array = h1, title = 'End of Year 1 (Steady state)')
plot_head(array = h2, title = 'End of Year 2 (Pumping)')
plot_head(array = h3, title = 'End of Year 3 (Recovery)')

### Plot observations in time

In [ ]:
# Define 3 observation locations and create time-series array for each
head_pump = []
head_obs1 = []
head_obs2 = []
head_obs3 = []
for i in range(nts1 + nts2 + nts3):
    head_pump.append(heads_all[i,0,0,pump1_cell])
    head_obs1.append(heads_all[i,0,0,obs_cells[0]])
    head_obs2.append(heads_all[i,0,0,obs_cells[1]])
    head_obs3.append(heads_all[i,0,0,obs_cells[2]])
print(len(head_obs1))

In [ ]:
# Plot time series head data at pump and 3 obs locations
fig, ax = plt.subplots()
fig.suptitle('Head profile at 3 obs bores in time')
ax.plot(head_pump, label = 'pump')
ax.plot(head_obs1, label = 'obs1')
ax.plot(head_obs2, label = 'obs2')
ax.plot(head_obs3, label = 'obs3')
#ax.set(ylim=(320, 350))
ax.set_xlabel('Timestep') 
ax.set_ylabel('Head (m)') 
ax.legend()